# indexing the processed text

[wikiextract](http://medialab.di.unipi.it/wiki/Wikipedia_Extractor) 
```shell
python ~/github/wikiextractor/WikiExtractor.py --processes 4 -ns 0,100,102,14,114 --json zhwikisource-20200301-pages-articles-multistream.xml

python ~/github/wikiextractor/WikiExtractor.py --processes 4 -ns 104 --json zhwikisource-20200301-pages-articles-multistream.xml
```

In [1]:
from pathlib import Path
from glob import glob
import json
import pandas as pd
import os
import sys

In [3]:
HOME = Path(os.environ["HOME"])

DATA = Path(HOME/"data"/"zhwiki")

In [15]:
all_files = glob(str(DATA/"text"/"*"/"*"))
print(len(all_files))
all_files[:5]

3237


['/Users/salvor/data/zhwiki/text/AX/wiki_73',
 '/Users/salvor/data/zhwiki/text/AX/wiki_87',
 '/Users/salvor/data/zhwiki/text/AX/wiki_80',
 '/Users/salvor/data/zhwiki/text/AX/wiki_74',
 '/Users/salvor/data/zhwiki/text/AX/wiki_89']

In [22]:
def file_to_dict(path):
    with open(path,"r") as f:
        while True:
            line = f.readline()
            if line:
                yield json.loads(line)
            else:
                break

In [29]:
def batch_df(path):
    return pd.DataFrame(list(d for d in file_to_dict(path)))

In [40]:
def hash_name(name):
    return hex(abs(hash(name)))

In [39]:
batch_df(all_files[0])

,id,url,title,text
0,311031,https://zh.wikisource.org/wiki?curid=311031,十先生奥論註 (四庫全書本)/續集卷15,十先生奥論註 (四庫全書本)/續集卷15\n\n<onlyinclude><poem> 欽...
1,311032,https://zh.wikisource.org/wiki?curid=311032,兩宋名賢小集 (四庫全書本)/卷263,兩宋名賢小集 (四庫全書本)/卷263\n\n<onlyinclude><poem> 欽定...
2,311033,https://zh.wikisource.org/wiki?curid=311033,十先生奥論註 (四庫全書本)/全覽1,十先生奥論註 (四庫全書本)/全覽1\n\n﻿\n﻿\n﻿\n﻿\n﻿\n﻿\n﻿\n﻿\n...
3,311034,https://zh.wikisource.org/wiki?curid=311034,龜巢稿 (四庫全書本)/卷14,龜巢稿 (四庫全書本)/卷14\n\n<onlyinclude><poem> 欽定四庫全書...
4,311035,https://zh.wikisource.org/wiki?curid=311035,兩宋名賢小集 (四庫全書本)/卷264,兩宋名賢小集 (四庫全書本)/卷264\n\n<onlyinclude><poem> 欽定...
...,...,...,...,...
62,311097,https://zh.wikisource.org/wiki?curid=311097,兩宋名賢小集 (四庫全書本)/卷299,兩宋名賢小集 (四庫全書本)/卷299\n\n<onlyinclude><poem> 欽定...
63,311098,https://zh.wikisource.org/wiki?curid=311098,兩宋名賢小集 (四庫全書本)/卷300,兩宋名賢小集 (四庫全書本)/卷300\n\n<onlyinclude><poem> 欽定...
64,311099,https://zh.wikisource.org/wiki?curid=311099,南齊文紀 (四庫全書本)/卷07,南齊文紀 (四庫全書本)/卷07\n\n<onlyinclude><poem> 欽定四庫全...
65,311100,https://zh.wikisource.org/wiki?curid=311100,兩宋名賢小集 (四庫全書本)/卷301,兩宋名賢小集 (四庫全書本)/卷301\n\n<onlyinclude><poem> 欽定...


Create saving destination

In [53]:
!mkdir -p {DATA/"cn_text"}

Saving text instead of json

In [56]:
def save_text(row):
    rid = row["id"]
    hashed = hash_name(row["title"])
    filename = f"{rid}_{hashed}.txt"
    with open(DATA/"cn_text"/filename,"w") as f:
        f.write(row["text"])
    return filename

In [62]:
for path in all_files:
    df = batch_df(path)
    df["filename"] = df.apply(save_text, axis=1)
    df[["id","url","title","filename"]].to_csv(DATA/"zhwikisource_title.csv",
                                               header=False,
                                               index=False,
                                               mode="a", )
    os.system(f"rm {path}")
    sys.stdout.write(f"\r{path}")

/Users/salvor/data/zhwiki/text/AZ/wiki_37

In [6]:
for i in list([0,1,2,3,4,5,6,7,8,9,"a","b","c","d","e","f"]):
    for j in list([0,1,2,3,4,5,6,7,8,9,"a","b","c","d","e","f"]):
        path = str(f"/Users/salvor/data/zhwiki/cn_text/*{i}{j}.txt")
        os.system(f"rm -rf {path}")
        sys.stdout.write(f"\r{path}")

/Users/salvor/data/zhwiki/cn_text/*ff.txt

In [3]:
df = pd.read_csv("/Users/salvor/data/cn_titles.csv")

In [5]:
for uuid in df.uuid:
    os.system(f"rm -rf /Users/salvor/data/cn_text/{uuid}.txt")